# Classification

Création des sets et labels de référence, de validation et de test

In [5]:
import numpy as np
import imageio

def array_to_list(array):
    L = []
    (l,k) = np.shape(array)
    for i in range(l):
        for j in range(k):
            L.append(array[i,j])
    return L    

letter_array =['B','C','D','E','F','G','H','I','J','K'];
base_ref = []
label_ref = []
for letter in range(0,10):
    for i in range(1,151):
        image_file_name = 'data_resized/' + letter_array[letter] + str(i) + '.bmp'
        image = imageio.imread(image_file_name)
        list_img = array_to_list(image)
        base_ref.append(list_img)
        label_ref.append(letter_array[letter])
label_ref = np.array([e for e in label_ref])       
base_ref = np.array([l for l in base_ref])

base_val = []
label_val = []
for letter in range(0,10):
    for i in range(151,201):
        image_file_name = 'data_resized/' + letter_array[letter] + str(i) + '.bmp'
        image = imageio.imread(image_file_name)
        list_img = array_to_list(image)
        base_val.append(list_img)
        label_val.append(letter_array[letter])
        
label_val = np.array([e for e in label_val])       
base_val = np.array([l for l in base_val])

base_test = []
label_test = []
for letter in range(0,10):
    for i in range(201,251):
        image_file_name = 'data_resized/' + letter_array[letter] + str(i) + '.bmp'
        image = imageio.imread(image_file_name)
        list_img = array_to_list(image)
        base_test.append(list_img)
        label_test.append(letter_array[letter])
        
label_test = np.array([e for e in label_test])       
base_test = np.array([l for l in base_test])

(500, 144)


Méthode des k plus proches voisins

In [3]:
def test_ppv(base_ref,label_ref,image,k):
    dist_image_ref = []
    for i in range(len(base_ref)):
        dist = np.linalg.norm(image - base_ref[i,:])
        dist_image_ref.append(dist)
    index_min_dist = np.argpartition(dist_image_ref,k)[0:k]
    class_min_dist = [label_ref[i] for i in index_min_dist]
    return class_min_dist

In [4]:
from collections import Counter

def class_maj(classes_list):
    c = Counter(classes_list)
    if len(c)>1:
        classes = c.most_common(2)
        if classes[0][1] == classes[1][1]:
            return -1
        return c.most_common(1)[0][0]
    else:
        return c.most_common(1)[0][0]

In [5]:
def matrice_confusion(base,label,k):
    conf_array = np.zeros((10,11),int)
    for i in range(len(base)):
        pred = class_maj(test_ppv(base_ref,label_ref,base[i,:],k))
        if pred == -1:
            conf_array[letter_array.index(label[i]),-1] +=1
        else:
            conf_array[letter_array.index(label[i]),letter_array.index(pred)] += 1
    return conf_array

In [6]:
def label_pred(base,label,k):
    label_pred = []
    test = []
    for i in range(len(base)):
        pred = class_maj(test_ppv(base_ref,label_ref,base[i,:],k))
        label_pred.append(pred)
    return label_pred

In [15]:
print(matrice_confusion(base_test[0:50],label_test[0:50],1))

[[14  0  3  2  0  3  8 19  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]]


In [13]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(label_test, label_pred(base_test,label_test,1)))

[[14  0  3  2  0  3  8 19  0  1]
 [ 3 29  0  1  5  2  0 10  0  0]
 [ 7  0 16  0  8  2  4 12  0  1]
 [ 0  2  0  8  5  0  0 35  0  0]
 [ 0  2  3  8 13  0  0 24  0  0]
 [ 2  1  1  1  1 15  1 27  0  1]
 [ 1  1  3  1  5  1 34  4  0  0]
 [ 0  0  0  2  1  0  0 47  0  0]
 [ 0  1  0  2 11  0  0 31  5  0]
 [ 1  2  2  6  4  0  2 27  0  6]]


In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

classifier = KNeighborsClassifier(n_neighbors=2,algorithm='brute')
classifier.fit(base_ref, label_ref)
label_pred = classifier.predict(base_test)
print(confusion_matrix(label_test, label_pred))
print(classification_report(label_test, label_pred))

[[44  0  3  0  0  3  0  0  0  0]
 [ 0 48  0  1  0  1  0  0  0  0]
 [ 5  0 42  0  0  1  2  0  0  0]
 [ 0 10  0 39  1  0  0  0  0  0]
 [ 0  1  0  2 47  0  0  0  0  0]
 [ 3  3  0  2  0 42  0  0  0  0]
 [ 5  0  0  0  0  0 44  0  0  1]
 [ 0  0  0  2  1  0  0 43  4  0]
 [ 0  0  0  0  1  1  0  6 41  1]
 [ 0  1  0  0  1  0 10  0  0 38]]
             precision    recall  f1-score   support

          B       0.77      0.88      0.82        50
          C       0.76      0.96      0.85        50
          D       0.93      0.84      0.88        50
          E       0.85      0.78      0.81        50
          F       0.92      0.94      0.93        50
          G       0.88      0.84      0.86        50
          H       0.79      0.88      0.83        50
          I       0.88      0.86      0.87        50
          J       0.91      0.82      0.86        50
          K       0.95      0.76      0.84        50

avg / total       0.86      0.86      0.86       500

